# Control PC with Hands

In [1]:
from aircontrol.utils import get_window, numlock_off
from aircontrol.const import BaseOptions, GestureRecognizer, GestureRecognizerOptions, VisionRunningMode, model_path
from aircontrol.params import window_aware_gestures
from aircontrol.models.midas import MidasModel
import mediapipe as mp
import cv2 as cv
import numpy as np
import time
import os

## Callback

In [2]:
def action_callback(result, output_image: mp.Image, timestamp_ms: int):
    global window_aware_gestures
    global key_pressed
    if not key_pressed:
        try:
            window, sub_window = get_window()
            sign = result.gestures[0][0].category_name
            if sign != "none" and sign != "":
                try:
                    func = window_aware_gestures[window][sub_window][sign]
                except KeyError:  # if action or window not specified in the dict
                    try:
                        func = window_aware_gestures[window][sign]
                    except KeyError:
                        func = window_aware_gestures["General"][sign]
                numlock_off()
                if callable(func):
                    func()
                else:
                    pos = result.hand_landmarks[0]
                    x_values = [d.x for d in pos]
                    xavg = np.mean(x_values)
                    if (
                        result.handedness[0][0].category_name == "Right"
                        and xavg >= 0.65
                    ):
                        func["Right"]()
                    elif (
                        result.handedness[0][0].category_name == "Left" and xavg <= 0.35
                    ):
                        func["Left"]()
                    else:
                        func["Center"]()
            key_pressed = True
        except:
            pass

## MediaPipe Configuration

In [3]:
options = GestureRecognizerOptions(
    base_options=BaseOptions(
        model_asset_path=os.path.join(
            os.getcwd(), model_path
        )
    ),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=action_callback,
)

recognizer = GestureRecognizer.create_from_options(options)

## Make detection easier with MiDaS

In [4]:
midas=MidasModel()

Using cache found in C:\Users\Andrea/.cache\torch\hub\intel-isl_MiDaS_master
c:\Users\Andrea\Documenti\aircontrol\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Andrea\Documenti\aircontrol\venv\lib\site-packages\timm\models\_factory.py:117: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(
Using cache found in C:\Users\Andrea/.cache\torch\hub\intel-isl_MiDaS_master


## The detection loop

In [ ]:
cam=cv.VideoCapture(0)
print('Starting...')
key_pressed=False
while True:
    success, frame=cam.read()
    frame=cv.flip(frame, 1)
    
    if not success:
        print("Camera Frame not available")
        continue
    
    adjusted_frame=midas.predict_adjust(frame)

    frame_mp=mp.Image(image_format=mp.ImageFormat.SRGB, data=adjusted_frame)
    
    timestamp_seconds=time.time()
    frame_timestamp_ms=int(timestamp_seconds * 1000)

    if not key_pressed:
        recognizer.recognize_async(frame_mp, frame_timestamp_ms)
    
    if key_pressed:
        key_pressed=False
    cv.imshow("Show Video", adjusted_frame)
    
    if cv.waitKey(20) & 0xff==ord('q'):
        cam.release()
        break

cam.release()
cv.destroyAllWindows()